## Train model identify if two images are the same turtle

### Import libraries

In [8]:
import os
import glob
import json
from os.path import join

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import random

from ShellRec.data_utils.prepare_photos import get_img_graph, split_graph
from ShellRec.model import TurtleDiffPool 
from ShellRec.dataset import TurtlePair 
from ShellRec.train import train

In [9]:
random.seed(42)

### Preprocessing photo files

In [10]:
all_train = get_img_graph(path = "../dataset/BoxTurtle", drop_p=[0.99,0])
split_graph(all_train, save_path = "../dataset")
_ = get_img_graph(path = "../dataset/BoxTurtle_holdout", 
                  file_to_save = "../dataset/BoxTurtle_holdout.json")

### Prepare torch dataset

In [11]:
## image transformations
transform_train = timm.data.create_transform(384, is_training = True, 
                                   auto_augment = "rand-m9-mstd0.5")


transform_test = transforms.Compose([
        transforms.Resize((384,384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[.5,.5,.5], 
                             std=[0.5, 0.5, 0.5])
])

train_set = TurtlePair(data_file='../dataset/train.json',transform=transform_train)
val_set = TurtlePair(data_file='../dataset/val.json',transform=transform_test)

# Set up datasets and dataloaders
train_loader = DataLoader( TurtlePair(data_file='../dataset/train.json', 
                                             transform=transform_train), 
                                             batch_size = 64)
val_loader = DataLoader( TurtlePair(data_file='../dataset/val.json', 
                                           transform=transform_test), 
                                           batch_size = 64)

### training

Get the model set up, use a pretrained model as backbone, and add a new head to it.

In [12]:
torch.hub.set_dir('../pretrained/')

In [13]:
#device = "cpu" #
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   

model = TurtleDiffPool('vit_base_patch16_384') # use a vit backbone
#model = TurtleDiff('resnet50') # use a resnet backbone
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
num_epochs = 20

Training loop

In [14]:
loss_list = train(model, optimizer, criterion, train_loader, val_loader, device, num_epochs,
      save_path = "../pretrained")

Epoch: 1/20


100%|██████████| 19/19 [09:01<00:00, 28.47s/it]


Epoch: 1/20, Training Loss: 0.5256


100%|██████████| 3/3 [01:11<00:00, 23.71s/it]


Epoch: 1/20, Validation Accuracy: 57.24%
Epoch: 2/20


...

Epoch: 19/20, Validation Accuracy: 91.45%
Epoch: 20/20


100%|██████████| 19/19 [08:51<00:00, 28.00s/it]


Epoch: 20/20, Training Loss: 0.3329


100%|██████████| 3/3 [01:11<00:00, 23.96s/it]

Epoch: 20/20, Validation Accuracy: 91.45%
